In [1]:
# librerias
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv('./data/users_web.csv')
#df = df.rename(columns={"Date":"date"})
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
#df = pd.read_csv('./data/users_web.csv', index_col='Date')
df = df.set_index('Date')
df.head(50)

,Users
Date,
2020-01-01,13
2020-01-02,17
2020-01-03,13
2020-01-04,4
2020-01-05,10
2020-01-06,29
2020-01-07,43
2020-01-08,27
2020-01-09,26


In [3]:
X = df["Users"].values
train = X[:-60]
test = X[-60:-30]

In [4]:
import itertools
p=q=range(0,15)
d = range(0,3)
pdq = list(itertools.product(p,d,q))
pdq[:6]

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 0, 5)]

In [5]:
results = []
for param in pdq:
    try:
        model_arima = ARIMA(order=param)
        model_arima_fit = model_arima.fit(train)
        results.append((param, model_arima_fit.aic()))
    except:
        continue

In [6]:
resultados_df = pd.DataFrame(results, columns=["ARIMA params", "AIC"]).sort_values("AIC")
resultados_df.head()

,ARIMA params,AIC
385,"(8, 1, 10)",6603.773128
383,"(8, 1, 8)",6606.196260
517,"(11, 1, 7)",6607.499386
430,"(9, 1, 10)",6608.093545
475,"(10, 1, 10)",6608.173619


In [7]:
best_model = ARIMA(order=(8,1,10))
best_model.fit(train)

ARIMA(order=(8, 1, 10))

In [8]:
predictions = best_model.predict(30)
print("mean_squared_error:", mean_absolute_percentage_error(test, predictions))

mean_squared_error: 0.19982096311838685


In [9]:
import pickle

In [12]:
# Guardar el modelo
import pickle
with open('finished_model.model', "wb") as archivo_salida:
    pickle.dump(best_model, archivo_salida)

In [13]:
# Para volver a leer el modelo
with open('finished_model.model', "rb") as archivo_entrada:
    pipeline_importada = pickle.load(archivo_entrada)
    
print(pipeline_importada)

 ARIMA(8,1,10)(0,0,0)[0] intercept
